In [6]:
import xarray as xr
import sqlite3
import pandas as pd

# List of NetCDF files and corresponding variable names
files = [('prediction_data/eastward_near_surface_wind/uas_Amon_EC-Earth3-CC_ssp245_r1i1p1f1_gr_20160116-20461216_v20210113.nc', 'u_wind'),
         ('prediction_data/northward_near_surface_wind/vas_Amon_EC-Earth3-CC_ssp245_r1i1p1f1_gr_20160116-20461216_v20210113.nc', 'v_wind'),
         ('prediction_data/precipitation/pr_Amon_EC-Earth3-CC_ssp245_r1i1p1f1_gr_20160116-20461216_v20210113.nc', 'precipitation'),
         ('prediction_data/near_surface_air_temperature/tas_Amon_EC-Earth3-CC_ssp245_r1i1p1f1_gr_20160116-20461216_v20210113.nc', 'temperature')
         ]

# Initialize an empty DataFrame
df_all = pd.DataFrame()

for file, variable in files:
    # Load NetCDF file
    ds = xr.open_dataset(file)

    # Convert to DataFrame and reset index to flatten the data
    df = ds.to_dataframe().reset_index()

    # Add a column for the variable name
    df['variable'] = variable

    # Concatenate the DataFrame with the main DataFrame
    df_all = pd.concat([df_all, df])

## Create a SQLite database
#conn = sqlite3.connect('prediction_data.db')
#
## Store the DataFrame in SQLite table
#df_all.to_sql('prediction_data', conn, if_exists='replace', index=False)
#
#conn.close()
#print('Database created successfully.')
    
ds = xr.open_dataset(files[0][0])
ds

<xarray.Dataset>
Dimensions:    (time: 372, bnds: 2, lat: 3, lon: 2)
Coordinates:
  * time       (time) datetime64[ns] 2016-01-16T12:00:00 ... 2046-12-16T12:00:00
  * lat        (lat) float64 37.54 38.25 38.95
  * lon        (lon) float64 357.2 357.9
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (time, lat, bnds) float64 ...
    lon_bnds   (time, lon, bnds) float64 ...
    uas        (time, lat, lon) float32 ...
Attributes: (12/46)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   0.0
    branch_time_in_parent:  0.0
    contact:                cmip6-data@ec-earth.org
    ...                     ...
    tracking_id:            hdl:21.14100/130ab123-1a11-4a02-aed7-ac177fbc3366
    variable_id:            uas
    variant_info:           forcing: Nat.Ant. Member generated from autosubmi...
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by EC-Earth-Consortium ...
    cmor_version:           3.5.0

In [7]:
df_all

,time,bnds,lat,lon,time_bnds,lat_bnds,lon_bnds,height,uas,variable,vas,pr,tas
0,2016-01-16 12:00:00,0,37.54378,357.187500,2016-01-01,37.192900,356.835938,10.0,2.400109,u_wind,NaN,NaN,NaN
1,2016-01-16 12:00:00,0,37.54378,357.890625,2016-01-01,37.192900,357.539062,10.0,2.875284,u_wind,NaN,NaN,NaN
2,2016-01-16 12:00:00,0,38.24553,357.187500,2016-01-01,37.894655,356.835938,10.0,2.725600,u_wind,NaN,NaN,NaN
3,2016-01-16 12:00:00,0,38.24553,357.890625,2016-01-01,37.894655,357.539062,10.0,3.238209,u_wind,NaN,NaN,NaN
4,2016-01-16 12:00:00,0,38.94728,357.187500,2016-01-01,38.596405,356.835938,10.0,3.576389,u_wind,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4459,2046-12-16 12:00:00,1,37.54378,357.890625,2047-01-01,37.894655,358.242188,2.0,NaN,temperature,NaN,NaN,281.452301
4460,2046-12-16 12:00:00,1,38.24553,357.187500,2047-01-01,38.596405,357.539062,2.0,NaN,temperature,NaN,NaN,279.754700
4461,2046-12-16 12:00:00,1,38.24553,357.890625,2047-01-01,38.596405,358.242188,2.0,NaN,temperature,NaN,NaN,280.112213
4462,2046-12-16 12:00:00,1,38.94728,357.187500,2047-01-01,39.298155,357.539062,2.0,NaN,temperature,NaN,NaN,280.425934


In [15]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('prediction_data.db')

# Create a cursor object to execute SQL queries
cursor = conn.cursor()

# Run your SQL query
query = "SELECT time, lat, lon, uas FROM prediction_data WHERE variable = 'u_wind' "
cursor.execute(query)

# Fetch all the rows returned by the query
rows = cursor.fetchall()

# Display the contents of the dataset
for row in rows:
    print(row)

# Close the cursor and the connection
cursor.close()
conn.close()


('2016-01-16 12:00:00', 37.54378, 357.1875, 2.400109052658081)
('2016-01-16 12:00:00', 37.54378, 357.890625, 2.875283718109131)
('2016-01-16 12:00:00', 38.24553, 357.1875, 2.725599765777588)
('2016-01-16 12:00:00', 38.24553, 357.890625, 3.2382094860076904)
('2016-01-16 12:00:00', 38.94728, 357.1875, 3.5763890743255615)
('2016-01-16 12:00:00', 38.94728, 357.890625, 4.332744598388672)
('2016-01-16 12:00:00', 37.54378, 357.1875, 2.400109052658081)
('2016-01-16 12:00:00', 37.54378, 357.890625, 2.875283718109131)
('2016-01-16 12:00:00', 38.24553, 357.1875, 2.725599765777588)
('2016-01-16 12:00:00', 38.24553, 357.890625, 3.2382094860076904)
('2016-01-16 12:00:00', 38.94728, 357.1875, 3.5763890743255615)
('2016-01-16 12:00:00', 38.94728, 357.890625, 4.332744598388672)
('2016-02-15 12:00:00', 37.54378, 357.1875, 1.4740517139434814)
('2016-02-15 12:00:00', 37.54378, 357.890625, 1.738176703453064)
('2016-02-15 12:00:00', 38.24553, 357.1875, 1.7153284549713135)
('2016-02-15 12:00:00', 38.24553, 3